<a href="https://colab.research.google.com/github/sajithrw/RetinaInspect-AI/blob/develop/RetinaInspect_AI_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DataSet**

Using dataset *Diabetic Retinopathy 2015 Data Colored Resized* from kaggle.com. These images are resized into 224x224 pixels so that they can be readily used with many pre-trained deep learning models.

All of the images are already saved into their respective folders according to the severity/stage of diabetic retinopathy.

**Below are image classes:**
*   No_DR
*   Mild
*   Moderate
*   Severe
*   Proliferate_DR

All the parameters of ````ImageDataGenerator```` uses default values to avoid preprocessing of images as they are already processed.

All the images are uploaded to ````RetinaInspect-AI```` folder in Google drive.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# specify the directory containing the labeled subdirectories
image_directory = '/content/drive/MyDrive/RetinaInspect-AI/'

# data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    shear_range=0,
    zoom_range=0,
    horizontal_flip=False,
    fill_mode='nearest'
)

batch_size = 32
train_generator = datagen.flow_from_directory(
    image_directory,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

**Model VGG16**

Load pre-trained VGG16 models and fine-tune it for identification ot Diabetic Retinopathy.

Uses ````Adam```` optimizer to train the model. Finally saves the model after training as ````DR_detection_model.h5````.

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_model = VGG16(weights='imagenet', include_top=False)

# custome layers for classification task
layer = base_model.output
layer = GlobalAveragePooling2D()(layer)
layer = Dense(1024, activation='relu')(layer)
predictions = Dense(5, activation='softmax')(layer) #5 classes for DR levels

# compile and train model with dataset
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

num_epochs = 10
model.fit(train_generator, epochs=num_epochs)

# save trained model
model.save('DR_detection_model.h5')

58889256/58889256 [==============================] - 2s 0us/step


NameError: ignored